In [6]:
import CCACSP
from preprocessing import *
from import_data import *
data_dir = 'DataCollection/outputs/'
ssvep_dir = data_dir + 'SSVEP/'
mi_dir = data_dir + 'MI/sess2/'
bci_dir1 = '../Documents/OpenBCI_GUI/Recordings/OpenBCISession_2023-02-28_12-03-49/'
bci_dir2 = '../Documents/OpenBCI_GUI/Recordings/OpenBCISession_2023-03-10_14-09-51/'
col_names = ['Fp1', 'O1', 'O2','C3','C4','P3','P4','Pz']
mi_files = get_files(mi_dir)
#mi_files = [mi_files[i] for i in range(len(mi_files)) if (i+1)%6!=0]
bci1_files = get_files(bci_dir1)
bci2_files = get_files(bci_dir2)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
for i, fi in enumerate(mi_files):
    print("{}: {}".format(i,fi))

0: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MILF.txt
1: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MIRF.txt
2: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MISw.txt
3: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MITy.txt
4: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_NOMI.txt
5: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_baseline.txt
6: DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_metadata.txt
7: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MILF.txt
8: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MIRF.txt
9: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MISw.txt
10: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MITy.txt
11: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_NOMI.txt
12: DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_baseline.txt
13: DataCollection/outp

In [9]:
def get_train_data(mi_file, col_names, bci_file):
    eeg, Fs = process_eeg(mi_file, col_names, bci_file, low=2)
    eeg_np = eeg.to_numpy()
    npts = 5*Fs
    num_trials = int(eeg_np.shape[0] / npts)
    train_data = np.array([eeg_np[i*npts:(i+1)*npts].reshape(8,npts) for i in range(num_trials)])
    return train_data

In [10]:
swim = get_train_data(mi_files[2], col_names, bci2_files[1])

In [11]:
typing = get_train_data(mi_files[3], col_names, bci2_files[1])

In [12]:
numFilt = 8
filers, clf = CCACSP.train(swim, typing, numFilt)

In [13]:
swim_test = get_train_data(mi_files[8], col_names, bci2_files[1])
typing_test = get_train_data(mi_files[9], col_names, bci2_files[1])

In [14]:
swim_test.shape, typing_test.shape

((3, 8, 1250), (3, 8, 1250))

In [15]:
output = CCACSP.test(swim_test[2], filers, clf)
output

array([0.])

In [16]:
def get_session_data(sess_num, bci_file):
    data = []
    #for fi in mi_files[5*sess_num:5*(sess_num+1)-1]:
    for fi in mi_files[7*sess_num:7*(sess_num)+4]:
        print(fi)
        data.append(get_train_data(fi, col_names, bci_file))
    class1 = np.concatenate(data)
    class2 = get_train_data(mi_files[7*(sess_num)+4], col_names, bci_file)
    return class1, class2

In [17]:
def get_all_data(num_sessions, bci_file):
    mi = []
    nomi = []
    for i in range(num_sessions):
        class1, class2 = get_session_data(i, bci_file)
        mi.append(class1)
        nomi.append(class2)
    mi = np.concatenate(mi)
    nomi = np.concatenate(nomi)
    minLen = min(len(mi), len(nomi))
    mi = mi[:minLen]
    nomi = nomi[:minLen]
    return mi, nomi 

In [18]:
mi, nomi = get_all_data(2, bci2_files[1])

DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MILF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MIRF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MISw.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:10:37.683875_MI_MITy.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MILF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MIRF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MISw.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MITy.txt


In [19]:
mi.shape, nomi.shape

((24, 8, 1250), (24, 8, 1250))

In [27]:
def get_train_test(mi, nomi, split=0.8):
    np.random.shuffle(mi)
    np.random.shuffle(nomi)
    splitIdx = int(split*len(mi))
    train_mi = mi[:splitIdx]
    test_mi = mi[splitIdx:]
    train_nomi = nomi[:splitIdx]
    test_nomi = nomi[splitIdx:]
    return train_mi, train_nomi, test_mi, test_nomi

In [73]:
def get_results(train_mi, train_nomi, test_mi, test_nomi,show_results=True):
    numFilt = 8
    filters, clf = CCACSP.train(train_mi, train_nomi, numFilt)
    mi_res = []
    nomi_res = []
    for i in range(len(test_mi)):
        mi_result = CCACSP.test(test_mi[i], filters, clf)
        mi_res.append(mi_result[0])
        #print("True class: MI, Predicted: {}".format(mi_result[0]))
        nomi_result = CCACSP.test(test_nomi[i], filters, clf)
        nomi_res.append(nomi_result[0])
        #print("True class: NOMI, Predicted: {}".format(nomi_result[0]))
    decode = {0: 'MI', 1:'NOMI'}
    mi_print = [decode[res] for res in mi_res]
    nomi_print = [decode[res] for res in nomi_res]
    acc = (np.sum([1-mi for mi in mi_res]) + np.sum([nomi for nomi in nomi_res])) / (len(mi_res) + len(nomi_res))
    if show_results:
        print('MI results: ', mi_print)
        print('NOMI results: ', nomi_print)
        print('Accuracy: ', acc)
    return acc

In [72]:
split = 0.6
train_mi, train_nomi, test_mi, test_nomi = get_train_test(mi, nomi, split=split)
get_results(train_mi, train_nomi, test_mi, test_nomi)

MI results:  ['NOMI', 'MI', 'NOMI', 'NOMI', 'MI', 'NOMI', 'MI', 'NOMI', 'MI', 'NOMI']
NOMI results:  ['NOMI', 'NOMI', 'NOMI', 'NOMI', 'NOMI', 'NOMI', 'NOMI', 'NOMI', 'MI', 'NOMI']
Accuracy:  0.65


In [78]:
def get_average_results(mi, nomi, split, num_tests):
    accs = []
    for i in range(num_tests):
        train_mi, train_nomi, test_mi, test_nomi = get_train_test(mi, nomi, split)
        accs.append(get_results(train_mi, train_nomi, test_mi, test_nomi, False))
    print('Average accuracy: ', np.mean(accs))

In [97]:
get_average_results(mi, nomi, split=0.8, num_tests=100)

Average accuracy:  0.5810000000000001


In [54]:
numFilt = 8
filers, clf = CCACSP.train(mi, nomi, numFilt)

In [120]:
mi_test, nomi_test = get_all_data(1, bci2_files[1])

DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MILF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MIRF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MISw.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MITy.txt


In [121]:
nomi_test = nomi_test[:12]

In [122]:
mi_test.shape, nomi_test.shape

((12, 8, 1250), (12, 8, 1250))

In [123]:
mi_res = []
nomi_res = []
for i in range(mi_test.shape[0]):
    mi_result = CCACSP.test(mi_test[i], filers, clf)
    mi_res.append(mi_result)
    print("True class: MI, Predicted: {}".format(mi_result[0]))
    nomi_result = CCACSP.test(nomi_test[i], filers, clf)
    nomi_res.append(nomi_result)
    print("True class: NOMI, Predicted: {}".format(nomi_result[0]))

True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 1.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 1.0
True class: NOMI, Predicted: 1.0


In [124]:
np.mean(mi_res)

0.16666666666666666

In [125]:
np.mean(nomi_res)

1.0